In [8]:
import os

To make sure the notebook can run smoothly, do the following:
1. Install all the required packages. However, if you are prompted to install "Crypto", DO NOT do "pip install Crypto"! DO "pip install pycryptodome" instead. Here are more details: https://medium.com/@rajparmar23801/no-module-named-crypto-707f75186a6a
2. Go to Firebase. Click the gear icon next to "Project Overview". Choose "Project Settings" - Service Accounts - Generate New Private Key. Put the generated json file in the same directory as this notebook, and change the "serviceAccount" attribute in the next code block to the name of the json file you just downloaded.

## Firebase Storage Download (Image & Audio Data)

In [8]:
from firebase import Firebase

config = {
  "apiKey": "8YOlQyD5MNnZlMl4hAvqYz3uePYTO9bmuS7s9mAo",
  "authDomain": "capstone-caf92.firebaseapp.com",
  "databaseURL": "https://databaseName.firebaseio.com",
  "storageBucket": "capstone-caf92.appspot.com",
  "serviceAccount": "capstone-caf92-firebase-adminsdk-uckm2-527095b9c7.json"
}

# connect to firebase
firebase = Firebase(config)

In [9]:
# connect to firebase storage
storage = firebase.storage()

In [26]:
all_files = storage.list_files()
# download the image and audio data to specific path
for file in all_files:
    directory = file.name.split("/")
    target_dir = "../../data/" + "/".join(directory[:-1]) + "/" #may modify path
    isExist = os.path.exists(target_dir)
    if isExist:
        if os.path.isfile("../../data/" + file.name): #may modify path
            continue
    else:
        os.makedirs(target_dir)
    storage.child(file.name).download("../../data/" + file.name) #may modify path

## Firestore Download (Labels & Other Text Data)

In [11]:
# pip3 install firebase-admin
import firebase_admin
import csv
from firebase_admin import credentials, firestore

In [13]:
cred = credentials.Certificate("capstone-caf92-firebase-adminsdk-uckm2-527095b9c7.json")
#firebase_admin.initialize_app(cred)

### List current collection names

In [14]:
db = firestore.client()
listCollections = []
for col in db.collections():
    listCollections.append(col)
    print(col.id)

1
Debugthermal
Feb1
Feb15-0
Feb15-1
Feb2
Feb222
Feb2f
Feb2ff
Feb8-1
Feb8-2
Feb8-3
Feb8-4
Feb8-68f
Feb8-68f-1
Feb8-hf-1
Gbcnb
Gghhf
Ghfgfjg
Hhhhello
Jan18
Jan18test001
Jan30
Jan30c
Jan30c2
Jan30c3
Qq
Rygcb


### Pull the target collection to CSV (change target_id)

In [15]:
target_id = "Feb8-1" #Change this to the collection you want (to limit read and prevent exceeding quota)
for col in listCollections:
    if col.id != target_id:
        continue
    #Check if target directory & file exists
    target_dir = "../../data/firestore/"
    isExist = os.path.exists(target_dir)
    #Audio part
    if isExist:
        if os.path.isfile(target_dir + col.id + "_audio_raw.csv") or os.path.isfile(target_dir + col.id + "_audio_labels.csv"):
            continue
    else:
        os.makedirs(target_dir)
    audio = col.document('audio').collections()
    for entry in audio:
        audioList = []
        for doc in entry.stream():
            audioList.append(doc.to_dict())
        field_names = list(audioList[0].keys())
        with open(target_dir + col.id + "_audio_" + entry.id + ".csv","w") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames = field_names)
            writer.writeheader()
            writer.writerows(audioList)
    # #Image part
    # if isExist:
    #     if os.path.isfile(target_dir + col.id + "_images_thermal.csv"):
    #         continue
    # else:
    #     os.makedirs(target_dir)
    # image = col.document('images').collections()
    # for entry in audio:
    #     imageList = []
    #     for doc in entry.stream():
    #         imageList.append(doc.to_dict())
    #     field_names = list(imageList[0].keys())
    #     with open(target_dir + col.id + "_images_" + entry.id + ".csv","w") as csvfile:
    #         writer = csv.DictWriter(csvfile, fieldnames = field_names)
    #         writer.writeheader()
    #         writer.writerows(imageList)
